In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# =========================
# 1. 数据加载
# =========================

train_path = "../liar_dataset/processed_dataset/processed_train.csv"
valid_path = "../liar_dataset/processed_dataset/processed_valid.csv"
test_path = "../liar_dataset/processed_dataset/processed_test.csv"

train_data = pd.read_csv(train_path)
valid_data = pd.read_csv(valid_path)
test_data = pd.read_csv(test_path)

# =========================
# 标签映射
# =========================
label_mapping = {
    "pants-fire": 0,
    "false": 1,
    "barely-true": 2,
    "half-true": 3,
    "mostly-true": 4,
    "true": 5
}

train_data["label"] = train_data["label"].map(label_mapping)
valid_data["label"] = valid_data["label"].map(label_mapping)
test_data["label"] = test_data["label"].map(label_mapping)

# =========================
# 2. Tokenizer & Padding
# =========================

max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_data["clean_statement"])

train_seq = tokenizer.texts_to_sequences(train_data["clean_statement"])
valid_seq = tokenizer.texts_to_sequences(valid_data["clean_statement"])
test_seq = tokenizer.texts_to_sequences(test_data["clean_statement"])

train_seq = pad_sequences(train_seq, maxlen=max_len)
valid_seq = pad_sequences(valid_seq, maxlen=max_len)
test_seq = pad_sequences(test_seq, maxlen=max_len)

# =========================
# 3. LSTM 模型结构优化
# =========================

model = models.Sequential([
    layers.Embedding(input_dim=max_words, output_dim=128),
    layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
    layers.Attention(),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.5),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(6, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# =========================
# 4. 模型训练
# =========================

train_labels = train_data["label"].values
valid_labels = valid_data["label"].values

early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-5)

history = model.fit(
    train_seq, train_labels,
    epochs=20,
    batch_size=64,
    validation_data=(valid_seq, valid_labels),
    callbacks=[early_stop, reduce_lr],
    verbose=2
)

# =========================
# 5. 可视化训练过程
# =========================

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss Curve')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy Curve')

plt.tight_layout()
plt.show()

# =========================
# 6. 模型评估与预测结果
# =========================

test_labels = test_data["label"].values
y_pred_prob = model.predict(test_seq)
y_pred = np.argmax(y_pred_prob, axis=1)

print("Classification Report:\n")
print(classification_report(test_labels, y_pred))

# 混淆矩阵
cm = confusion_matrix(test_labels, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# =========================
# 7. 导出预测结果
# =========================

result_df = pd.DataFrame({
    "id": test_data["id"],
    "true_label": test_labels,
    "predicted_label": y_pred
})

result_df.to_csv("./predict_result_lstm.csv", index=False)

print("✅ 预测结果已保存至 ./predict_result_lstm.csv")


ModuleNotFoundError: No module named 'tensorflow.keras'